In [35]:
import numpy as np
import matplotlib.pyplot as plt
#import uproot
import pandas as pd
import scipy.optimize as opt
import math
import statistics as st
import os
import PyQt5
from numba import jit
import time
import numba
from numba.typed import List
from numba import uint16, intc, uint32
from array import array


print(os.getcwd())
os.chdir('F:/LANL/')
datadir = 'sample_data/'
folder = 'La_sample/'
run_num = "11143" 
print(os.getcwd() + folder)
statefileloc = 'F:\LANL\SF_Norm_files\TR_R_expected_avgs_stds_afterclip.csv'
savefilename = 'SF_Norm_files/'+folder+run_num

%matplotlib qt

start = time.time()

#chan_enab = [25,12,24,26,27,28] ## testing only!!
chan_enab = [25,26,27,28]

#@jit(nopython = True)
# read_data = np.array([])
# fileLength = np.array([])

read_data = []
fileLength = []

for el in chan_enab:
    f = open(datadir + folder + 'run' + run_num + "_ch" + str(el) + ".bin", 'rb')
    read_data.append(np.fromfile(file=f, dtype=np.uint16))
    f.close()
    fileLength.append(len(read_data[-1]))

# def test():
#     for el in chan_enab:
#         f = open(folder + 'run' + run_num + "_ch" + str(el) + ".bin", 'rb')
#         read_data.append(np.fromfile(file=f, dtype=np.uint16))
#         #chdat = np.fromfile(file=f, dtype=np.uint16)
#         #print((len(chdat)))
#         #read_data = np.vstack(read_data,chdat)
#         #print(len(read_data))
#         f.close()
#         #print((read_data))
#         fileLength.append(len(read_data[-1]))
#         #np.append(fileLength,len(read_data[-1]))
#         #read_data = np.asarray(read_data)
#     return read_data, fileLength


# #np.asarray(fileLength)
# test()
# #print(read_data)
# #print(type(fileLength))
# #print(type(read_data))
# #np.asarray(read_data)

fileLength = np.asarray(fileLength)
# read_data = np.asarray(read_data, dtype = object)  ## cannot do np array for sorting because He and SF are different sizes

print('processing data: ' + folder + 'run' + run_num)
print('saving to state & norm information to ' + savefilename)
end = time.time()
print(end-start)
# print(read_data)

F:\LANL
F:\LANLLa_sample/
processing data: La_sample/run11143
saving to state & norm information to SF_Norm_files/La_sample/11143
1.1779282093048096


Store the big header for each channel in arrays

In [37]:
BoardID = []
recordLength = []
numSamples = []
eventCounter = []
decFactor = []
chanDec = []
postTrig = []
groupStart = []
groupEnd = []
timestamp= []
sizeFirstEvent = []
TTT = []

targetDict = {0: "La", 1: "Tb2O3", 2: "Yb2O3", 3: "Sm2O3", 4: "Er2O3", 5: "Ho2O3", 6: "other"}
foilDict = {0: "TBD", 1: "TBD", 2: "TBD", 3: "TBD", 4: "TBD", 5: "TBD", 6: "other"}

target=(read_data[0][5]&0x00F0)>>4
foil=read_data[0][5]&0x000F
targetFlag = read_data[0][5]>>8&1
foilFlag = read_data[0][5]>>9&1
spinFiltFlag = read_data[0][5]>>10&1
spinFlipFlag = read_data[0][5]>>11&1
shutterFlag = read_data[0][5]>>12&1
facilityTrigFlag = read_data[0][5]>>13&1

if targetFlag:
    target=targetDict[(read_data[0][5]&0x00F0)>>4]
    
else:
    target = "empty"
    
    
if foilFlag:
    foil=foilDict[read_data[0][5]&0x000F]
else:
    foil = "empty"

for i in range(0,len(chan_enab)):
    BoardID.append(read_data[i][9]>>8)
    recordLength.append(((read_data[i][9]&0x00FF)<<16)+read_data[i][8])
    numSamples.append(((read_data[i][11]&0x00FF)<<16)+read_data[i][10])
    eventCounter.append(read_data[i][6]+(read_data[i][7]<<16))
    BoardID.append(read_data[i][9]>>8)  
    decFactor.append(read_data[i][11]>>8)
    chanDec.append(read_data[i][13]>>8)
    postTrig.append(read_data[i][15]>>8)
    groupStart.append(((read_data[i][13]&0x00FF)<<16)+read_data[i][12])
    groupEnd.append(((read_data[i][15]&0x00FF)<<16)+read_data[i][14])
    
    timestamp.append(read_data[i][16]+(read_data[i][17]<<16)+(read_data[i][18]<<32)+(read_data[i][19]<<40))  
    sizeFirstEvent.append(read_data[i][0]+(read_data[i][1]<<16))
    TTT.append(read_data[i][2]+(read_data[i][3]<<16)+(read_data[i][4]<<32))
    
    print("For channel " + str(chan_enab[i]) + ", BoardID is " + str(BoardID[i])
          + "; record length is " + str(recordLength[i]) + "; num Samples is " 
          + str(numSamples[i]) + "; event counter is " + str(eventCounter[i]) + "; dec factor is " + str(decFactor[i]) + "; chan dec is " 
          + str(chanDec[i]) + "; postTrig is " + str(postTrig[i]) + "; group start is " + str(groupStart[i]) + "; group end is " + str(groupEnd[i])
          + "; epoch time is " + str(timestamp[i]) +  "; first event size is " + str(sizeFirstEvent[i]) + "; and ETTT is " + str(TTT[i]) + "\n")

# BoardID = np.asarray(BoardID) 
# recordLength = np.asarray(recordLength)
numSamples = np.asarray(numSamples)
# decFactor = np.asarray(decFactor)
# chanDec = np.asarray(chanDec)
# postTrig = np.asarray(postTrig)
# groupStart = np.asarray(groupStart)
# groupEnd = np.asarray(groupEnd)
# timestamp = np.asarray(timestamp)
# sizeFirstEvent = np.asarray(sizeFirstEvent)
eventCounter = np.asarray(eventCounter)
TTT = np.asarray(TTT)

print("Target is " + target)
print("Foil is " + foil)
print("Shutter is open: " + str(bool(shutterFlag)))
print("Facility t0 is on: " + str(bool(facilityTrigFlag)))
print("Spin flipper is on: " + str(bool(spinFlipFlag)))
print("Spin filter is on: " + str(bool(spinFiltFlag)))
print("Target is present: " + str(bool(targetFlag)))
print("Foil is present: " + str(bool(foilFlag)))


For channel 25, BoardID is 31; record length is 45000; num Samples is 45000; event counter is 20000; dec factor is 32; chan dec is 1; postTrig is 80; group start is 0; group end is 45000; epoch time is 1706060412; first event size is 90040; and ETTT is 561986561

For channel 26, BoardID is 31; record length is 45000; num Samples is 351; event counter is 20000; dec factor is 32; chan dec is 128; postTrig is 80; group start is 0; group end is 45000; epoch time is 1706060412; first event size is 742; and ETTT is 561986561

For channel 27, BoardID is 31; record length is 45000; num Samples is 351; event counter is 20000; dec factor is 32; chan dec is 128; postTrig is 80; group start is 0; group end is 45000; epoch time is 1706060412; first event size is 742; and ETTT is 561986561

For channel 28, BoardID is 31; record length is 45000; num Samples is 351; event counter is 20000; dec factor is 32; chan dec is 128; postTrig is 80; group start is 0; group end is 45000; epoch time is 1706060412

Determine the time axis for each channel

In [40]:
preTime = []
startTime = []
endTime = []
resolution = []
xs = [] 

for i in range(0,len(chan_enab)):
    preTime.append((100-postTrig[i])*recordLength[i]/100)
    startTime.append((-1*preTime[i]*16*decFactor[i] + groupStart[i]*16*decFactor[i]))
    endTime.append((-1*preTime[i]*16*decFactor[i] + groupEnd[i]*16*decFactor[i]))
    resolution.append(16*chanDec[i]*decFactor[i])
    print("Pretime for channel", chan_enab[i],"is " + str(preTime[i]) + "; start time is " + str(startTime[i]) + "; end time is " + str(endTime[i]) 
          + "; resolution is " + str(resolution[i]) + "ns")
    xs.append(np.arange(startTime[i],(numSamples[i])*resolution[i]+startTime[i], resolution[i]))

#np.asarray(preTime)
#np.asarray(startTime)
#np.asarray(endTime)
#np.asarray(resolution)
#xs = np.asarray(xs) 
    
#print(len(xs[0]))
#print(preTime[0]/512)

Pretime for channel 25 is 9000.0; start time is -4608000.0; end time is 18432000.0; resolution is 512ns
Pretime for channel 26 is 9000.0; start time is -4608000.0; end time is 18432000.0; resolution is 65536ns
Pretime for channel 27 is 9000.0; start time is -4608000.0; end time is 18432000.0; resolution is 65536ns
Pretime for channel 28 is 9000.0; start time is -4608000.0; end time is 18432000.0; resolution is 65536ns


In [43]:
print(read_data)
print(len(read_data[0]))
print(np.asarray(read_data, dtype = object))

[array([24504,     1, 15361, ...,  2200,  2203,  2206], dtype=uint16), array([  742,     0, 15361, ...,  2040,  2040,  2040], dtype=uint16), array([  742,     0, 15361, ...,  2057,  2057,  2057], dtype=uint16), array([  742,     0, 15361, ...,  2064,  2064,  2064], dtype=uint16)]
225030014
[array([24504,     1, 15361, ...,  2200,  2203,  2206], dtype=uint16)
 array([  742,     0, 15361, ...,  2040,  2040,  2040], dtype=uint16)
 array([  742,     0, 15361, ...,  2057,  2057,  2057], dtype=uint16)
 array([  742,     0, 15361, ...,  2064,  2064,  2064], dtype=uint16)]


In [45]:

start=time.time()

if chan_enab[0] != 25:
    #print('No 3He. Cannot normalize')
    raise Exception('3He not in first channel loaded. Cannot normalize')
else:
    pass

@jit(nopython = True)
def dataread(data, channels, fileLen, numSamps):
    #ys_arr = []
    numRuns = int((fileLen[0]-20-numSamps[0])/(numSamps[0]+6)+1)
    print(numRuns)
    ys_arr = np.zeros((len(channels), numRuns,numSamps[0]), dtype=np.uint16)
    ETTT_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    eventcount_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    for i in range(0,len(channels)):
        eventCount = 0
        byteCounter = 0
            #byte counter is really 2bytecounter, lol
        while byteCounter < fileLen[i]:
            if byteCounter == 0:
                ETTT_arr[i]=TTT[i]
                #ETTT_arr[i].append(TTT[i])
                eventcount_arr[i]=(eventCounter[i])
                byteCounter = 20
            else:
                ETTT_arr[i]=(data[i][byteCounter]+(data[i][byteCounter+1]<<16)+(data[i][byteCounter+2]<<32))
                eventcount_arr[i]=(data[i][byteCounter+4]+(data[i][byteCounter+5]<<16))
                byteCounter += 6
            for j in range(0, numSamps[i]):
                #if j == 0:
                    #ys_arr[i].append([])
                #print(byteCounter)
                ys_arr[i][eventCount][j]=data[i][byteCounter]
                byteCounter += 1
            eventCount += 1
    return ys_arr, ETTT_arr, eventcount_arr

#start=time.time()
ys_arrHe, ETTT_arrHe, eventcount_arrHe  = dataread(read_data, np.array([25]), fileLength, numSamples) ##hardcoded channel 25 for He
ys_arr, ETTT_arr, eventcount_arr  = dataread(read_data[1:], np.array([26,27,28]), fileLength[1:], numSamples[1:]) ##hardcoded channels for coils
#ys_arrtest, ETTT_arr, eventcount_arr  = dataread(read_data[1:], [12,24], fileLength[1:], numSamples[1:]) ##testing only!

end = time.time()
print(end-start)

ETTT_arr = np.vstack([ETTT_arrHe,ETTT_arr]) ## ordering makes sure that first array of new ETTT_arr is ETTT_arr of He
eventcount_arr = np.vstack([eventcount_arrHe,eventcount_arr])
print(len(eventcount_arr))
# print('He:')
# print(len(ys_arrHe))
# print(len(ys_arrHe[0]))
#print(len(ys_arr[0]))
#print(len(ys_arr[0][0]))


5000
5000
2.857494592666626
4


Put ADC values in arrays for each channel (one array per event, an array of events per channel) and put the miniheader information in an array

Calculate the time difference between each event within a file - used to check for dropping pulses. It seems that if we make the record window 49.152 ms long, we miss every other pulse (at 20 Hz). This is not that surprising - we presumably will not need a lot of data (or any) with the full 50 ms time window.

In [48]:
timeDif=[]
for i in range(0,len(chan_enab)):
    timeDif.append([])
    for j in range(len(ETTT_arr[i])-1):
        timeDif[i].append((ETTT_arr[i][j+1]-ETTT_arr[i][j])*8)
    print("Min time difference for channel", chan_enab[i], "is", min(timeDif[i]), "ns")
    print("Max time difference for channel", chan_enab[i], "is", max(timeDif[i]), "ns \n")
#print(timeDif)

Min time difference for channel 25 is 0 ns
Max time difference for channel 25 is 0 ns 

Min time difference for channel 26 is 0 ns
Max time difference for channel 26 is 0 ns 

Min time difference for channel 27 is 0 ns
Max time difference for channel 27 is 0 ns 

Min time difference for channel 28 is 0 ns
Max time difference for channel 28 is 0 ns 



In [49]:
#switchs = [31, 74, 117, 160, 203, 246, 289, 332]
#%matplotlib notebook
baseL = 0
baseRCoil = int(((preTime[1]-groupStart[1])*0.70)/chanDec[1])
baseRHe = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])
numRuns = int((fileLength[0]-20-numSamples[0])/(numSamples[0]+6)+1)

if numSamples[0] != 45000:
    print('He channel wrong size')
    raise
elif numSamples[1] != 351:
    print('Coil channel wrong size')
    raise
elif numSamples[2] != 351:
    print('Coil channel wrong size')
    raise
    
sums = []
ys_basesub = [] 
#legend =  ['He']
legend =  ['LO', 'TR', 'R']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
transitions = ['111->101', '101->100', '100->110', '110->101','101->110','110->111','111->100','100->111']
switchpulses = np.arange(255,614,45) ##found these pulses which correspond to states 0-8. Change p to plot them
#print(switchpulses)
p=0

s = switchpulses[p]
# s = 255
t=s+1

start = time.time()

## dont know why this is so slow ##
def plotter(ys, xs, baseR, numpoints):
    tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=float)
    for i in range((len(ys))):
        #tempys_basesub = []
        #tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=float)
        #tempsums =[]
        for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
            tempys_basesub[i][pulse]=np.subtract(ys[i][pulse], np.mean(ys[i][pulse][baseL:baseR]))
        for j in range(s, t): ## plot only interested pulses
            #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
            #print(sum(ys_basesub[i][j])) 
            plt.plot(xs[i], tempys_basesub[i][j] , label=legend[i]) #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
            plt.axvline(xs[0][baseL], ls = '--')
            plt.axvline(xs[0][baseR], ls = '--')
            #plt.axvline(xs[0][int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])], ls = '--', c ='m')
            plt.axvline(xs[0][baseR+5], ls = '--', c ='r') ## BaseR+5 line marks the beginning of the integral, until the end of samples.
#             plt.title('SF state transition' + transitions[p]) 
#             plt.xlabel("time from trigger (ns)")
#             plt.ylabel("ADC")
            plt.legend()
            
#plotter(ys_arrHe, xs, baseRHe,numSamples) ##plot 3He
#plotter(ys_arr, xs[1:], baseRCoil, numSamples[1:]) ##plot coils

#@jit(nopython = True) ## Actually JIT seems to be slower here!
def basesub_sum(ys, baseR, numpoints): ## for coils, could be used for He but below does that
    tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=np.float64)
    tempsums = np.zeros((len(ys), numRuns), dtype=np.float64)
    for i in range((len(ys))):
        for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
            tempys_basesub[i][pulse]=np.subtract(ys[i][pulse], np.mean(ys[i][pulse][baseL:baseR]))
            tempsums[i][pulse] = np.sum(tempys_basesub[i][pulse][baseR+5:-1])
    return tempys_basesub, tempsums
        
#ys_basesub, sums = basesub_sum(ys_arrtest, xs[1:3], baseRCoil, numSamples[1:3]) ##testing only!!
ys_basesub, sums = basesub_sum(ys_arr, baseRCoil, numSamples[1:])

@jit(nopython = True) ## separate function for He because this checks every point, can take a while
def basesub_normHe(ys, baseRegion, intgrRegion):
    tempys_basesub = np.zeros((len(ys), numRuns,45000), dtype=np.float64) #hardcode numSamples[0] = 45000
    tempsums = np.zeros((len(ys), numRuns), dtype=np.float64)
    for i in range((len(ys))): ## i is pretty much always 0 for 3He. Left it general.
        for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
            for j in range(intgrRegion[0]+1000, intgrRegion[1]): ## checking for saturation in He channel. restrict to slightly smaller range
                if ys[i][pulse][j] > 4060:  ## cutoff adc value (real is 4092)
                    print(('3He is saturating in normalization region at pulse,point: ' + str(pulse) + ', '+ str(j)))
                    raise
                else:
                    pass
            tempys_basesub[i][pulse]=np.subtract(ys[i][pulse], np.mean(ys[i][pulse][baseRegion[0]:baseRegion[1]]))
            tempsums[i][pulse] = np.sum(tempys_basesub[i][pulse][intgrRegion[0]:intgrRegion[1]])
            ## need to investigate adc saturation point
    return tempys_basesub, tempsums

baseRHe = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0]) #redefined for clarity

HeBaseReg = np.array([0, baseRHe])
HeIntgrReg = np.array([baseRHe+700, 15999]) ## hardcoded begin/end region for integral over NaI and 6Li regions
ys_basesubHe, HeNorms = basesub_normHe(ys_arrHe, HeBaseReg, HeIntgrReg)
print(HeNorms)
## got rid of xs in basesub, don't think we need them as an input 06.10.24

end = time.time()
print('plotting and/or base subtraction time: ' + str(end-start))            

#print(len(sums))
#print(sumsHe[0][20:30])
#print(sums[1][20:30])

[[6523310.99793719 6568454.51079368 6577308.85444469 ... 6468414.91380991
  6346348.03984072 6379080.51206277]]
plotting and/or base subtraction time: 3.6943199634552


In [50]:
#print(ys_arrHe[:][0])
#print(xs[0][17980:17990])
print(ys_arrHe[0][4][17970:18000])
print(ys_arrHe[0][4][17970:17990]+203)
print(numSamples[1:3])
print(baseRCoil)
#print(len(ys_arr[1:3][0][0]))
print(len(xs[1]))
# print(len(ys_arrtest))
# print(len(ys_arrtest[0][0]))
# print(len(ys_arrtest[1][0]))
print(len(HeNorms[0]))
print((HeNorms[0]))
print((HeNorms[0])/100000)

[3883 3882 3891 3887 3878 3885 3890 3886 3886 3894 3894 3898 3898 3894
 3891 3892 3898 3896 3891 3894 3895 3889 3887 3896 3891 3895 3894 3890
 3891 3899]
[4086 4085 4094 4090 4081 4088 4093 4089 4089 4097 4097 4101 4101 4097
 4094 4095 4101 4099 4094 4097]
[351 351]
49
351
5000
[6523310.99793719 6568454.51079368 6577308.85444469 ... 6468414.91380991
 6346348.03984072 6379080.51206277]
[65.23310998 65.68454511 65.77308854 ... 64.68414914 63.4634804
 63.79080512]


In [53]:
## for plotting purposes only. Leave commented unless testing

# Heregion = [baseRHe+700, 15999] ## 15999 is the right bin for right side of He integral
# pulse = 0

# plt.plot(xs[0],ys_basesubHe[0][pulse], label='3He base sub')
# plt.plot(xs[1],ys_basesub[0][pulse], label='NaI') 
# plt.plot(xs[2],ys_basesub[1][pulse], label='6Li') 


# plt.title('3He') 
# plt.xlabel("time from trigger (ns)")
# plt.ylabel("ADC")
# print(len(xs[0]))
# print(len(ys_arrHe[0][0]))

# plt.axvline(xs[0][baseRHe], ls = '--', c ='r')
# plt.axvline(xs[0][Heregion[0]], ls = '--')
# plt.axvline(xs[0][Heregion[1]], ls = '--')
# #plt.axvline(xs[1][-1], ls = '--', c ='g')
# #plt.axhline(4060, ls = '--')
# #plt.axvline(xs[0][17000], ls = '--', c ='r')

# #plt.text(1.5e7, 1200, 'test', fontsize=12)
# plt.legend()
# plt.show()

In [54]:
## testing

# expectedSumsTR = [sums[2][switchpulses[0]], sums[2][switchpulses[1]], sums[2][switchpulses[2]], sums[2][switchpulses[3]], sums[2][switchpulses[4]], sums[2][switchpulses[5]], sums[2][switchpulses[6]], sums[2][switchpulses[7]]]

# AllSwitches = []

# for expectedsum in expectedSumsTR:
#     diff_arr = np.absolute(expectedsum - sums[2])
#     #print(len(diff_arr))
#     found_sums =[]
#     #nearest_i = diff_arr.argmin()
#     for j in range(0,len(diff_arr)):
#         if diff_arr[j] < 300: ## 300 seems like a good number, except for trans 1, has to have a lower std, use original
#             found_sums.append(j)
#             #print(j)
#     AllSwitches.append(np.array(found_sums))
    
# for i in range(len(AllSwitches)):
#     print(len(AllSwitches[i]))
# print((AllSwitches))

In [55]:
start = time.time()

statefile = pd.read_csv(statefileloc)
print(statefile)
transitions = statefile['transition'].to_numpy() ## for some reasons "averages" and "std dev" need a space before them
expectedSumsTR_R = statefile[' averages'].to_numpy()
expectedStdsTR_R = statefile[' standard dev'].to_numpy()

AllSwitches = []
tolerance = 1000 ## see comments below

## can't use pre-existing np array because usually one array of unequal length
for i in range(len(expectedSumsTR_R)):
    diff_arr = np.absolute(np.add(sums[1],sums[2]) - expectedSumsTR_R[i])
    #print(len(diff_arr))
    found_sums =[]
    #nearest_i = diff_arr.argmin()
    for j in range(len(diff_arr)):
#         if diff_arr[j] < expectedStdsTR_R[i]*3: ## within 3 standard deviations of its respective std
## using std didn't work for La. Maybe just get rid of it and use a constant value...
        if diff_arr[j] < tolerance: ## this uses a constant "tolerance"
            found_sums.append(j)
    AllSwitches.append(np.array(found_sums))

#         #found_sums = np.diff(np.absolute(diff_arr[j]) < 20, prepend=False)
#         #print(diff_arr[j])
#         #print(found_sums)
#         #xing = np.argwhere(thresh_xing)[:,0]

# #print(diff_arr)
for i in range(len(AllSwitches)):
    print(len(AllSwitches[i]))
# print((AllSwitches))
# #print(len(sums))
# #print(expectedSumsTR)
# #print(len(ys_arr[1]))
# #plt.hist(sums, 100)
# #plt.show()

print(AllSwitches)

end = time.time()
print(end-start)

   transition    averages   standard dev
0    111->101  118730.181         37.050
1    101->100 -116769.448          7.521
2    100->110  339703.714         84.664
3    110->101   65919.200         47.375
4    101->110   27057.732         42.419
5    110->111 -177115.684         97.496
6    111->100  -66041.496         25.830
7    100->111  -26887.357         65.417
14
14
14
14
14
14
14
13
[array([  15,  375,  735, 1095, 1455, 1815, 2175, 2535, 2895, 3255, 3615,
       3975, 4335, 4695]), array([  60,  420,  780, 1140, 1500, 1860, 2220, 2580, 2940, 3300, 3660,
       4020, 4380, 4740]), array([ 105,  465,  825, 1185, 1545, 1905, 2265, 2625, 2985, 3345, 3705,
       4065, 4425, 4785]), array([ 150,  510,  870, 1230, 1590, 1950, 2310, 2670, 3030, 3390, 3750,
       4110, 4470, 4830]), array([ 195,  555,  915, 1275, 1635, 1995, 2355, 2715, 3075, 3435, 3795,
       4155, 4515, 4875]), array([ 240,  600,  960, 1320, 1680, 2040, 2400, 2760, 3120, 3480, 3840,
       4200, 4560, 4920]), array(

In [56]:
## testing sorting with pandas dataframe
transitions = ['111->101', '101->100', '100->110', '110->101','101->110','110->111','111->100','100->111']

#print(len(AllSwitches[3]))
#transition = ## change this to change transition

transitionSumsTR = []
transitionSumsR = []
transitionTR_RAvgs = []
transitionTR_Rstds = []
transitionSumsTR_R = []

for i in range(0,len(transitions)):
    tempTR = []
    tempR = []
    for j in range(0,len(AllSwitches[i])):
        #print(sums[1][AllSwitches[transition][i]])
        tempTR.append(sums[1][AllSwitches[i][j]])
        tempR.append(sums[2][AllSwitches[i][j]])
    transitionSumsTR.append(tempTR)
    transitionSumsR.append(tempR)
    transitionSumsTR_R.append(np.add(tempTR,tempR))
    transitionTR_RAvgs.append(np.average(np.add(tempTR,tempR)))
    transitionTR_Rstds.append(np.std(np.add(tempTR,tempR)))

print(len(transitionTR_RAvgs))

cols = ['transition', 'transition_locations', 'sumsTR_R', 'TR_R_avgs', 'TR_R_stds']
transSumsData = [transitions, AllSwitches, transitionSumsTR_R, transitionTR_RAvgs, transitionTR_Rstds]

#cols = ['transition', 'TR_R_avgs', 'TR_R stds']
#transSumsData = [transitions, transitionTR_RAvgs, transitionTR_Rstds]
print(len(transSumsData))
df_SF = pd.DataFrame({cols[0]: transSumsData[0],            
                   cols[1]: transSumsData[1],
                   cols[2]: transSumsData[2],
                   cols[3]: transSumsData[3],
                   cols[4]: transSumsData[4]})
print('original dataframe to work with, 7 transitions and their associated locations and sums: ')
print(df_SF)
print(np.get_printoptions()['threshold'])
np.set_printoptions(threshold=np.inf)

with pd.option_context('display.max_rows', None,
                      'display.max_columns', None,
                      'display.precision', 3,
                      ):
   print(df_SF.explode(['transition_locations', 'sumsTR_R']))
print('original dataframe is exploded so that transition_locations and associated sum is unpacked. Indices of state are kept as "nickname":')    
print('save Averages and Stds for future use')

#df_SF.to_csv(statefileloc, columns = ["TR_R_avgs", "TR_R_stds"]) ## not sure if i need to save this info
#np.savetxt("E:/LANL/analysis.csv", added_pulses[1], delimiter=",")

df_SF = df_SF.explode(['transition_locations', 'sumsTR_R']).reset_index().rename(columns={'index' : 'nicknames'}) #turn the 'index' of the exploded df_SF into a column, then reassign indices
#df_SF = df_SF.sort_values(by = ['transition_locations'])
print(df_SF)

print('now sort by the transition location and rearrange dataframe indices, only matters for looping (??):')   
#df_SF.sort_values(by=['transition_locations'])
df_SF = df_SF.sort_values(by=['transition_locations'])
df_SF = df_SF.reset_index(drop=True)
#print('rearrange dataframe indices, only matters for looping (??)')
print(df_SF)
                    
#with pd.option_context('display.max_rows', None,
#                       'display.max_columns', None,
#                       'display.precision', 3,
#                       ):

for ind in df_SF.index[:-1]:
    #print(ind)
    print('transition: '+ str(df_SF['nicknames'][ind]) + ' location: ' + str(df_SF['transition_locations'][ind]))
    #print(df_SF['nicknames'][ind])
    if (df_SF['nicknames'][ind+1])-1 != df_SF['nicknames'][ind]: ## if next transition 'nickname' is not next in sequence, failure
        if (df_SF['nicknames'][ind+1])-1 == -1: ## special condition for end of sequence where (0-1) != 7
            if (df_SF['transition_locations'][ind+1]-df_SF['transition_locations'][ind]) != 45: ## changed to 45 pulses!
                print('# pulses error: ' + str(df_SF['transition_locations'][ind+1]-df_SF['transition_locations'][ind]))
            if (df_SF['transition_locations'][ind+1]-df_SF['transition_locations'][ind]) == 45:
                print('# pulses correct, end of sequence')
        else:
            print('sorting failure, ' + str((df_SF['nicknames'][ind+1])-1) + '!=' + str(df_SF['nicknames'][ind]))
    elif (df_SF['nicknames'][ind+1])-1 == df_SF['nicknames'][ind]:
        #print('sorting good')
        #print('sorting good, ' + str((df_SF['nicknames'][ind]+1)-1) + '==' + str(df_SF['nicknames'][ind]))
        #pass
        #print(df_SF['transition_locations'][ind+1])
        #print(df_SF['transition_locations'][ind])
        if (df_SF['transition_locations'][ind+1]-df_SF['transition_locations'][ind]) != 45:
            print('# pulses error: ' + str(df_SF['transition_locations'][ind+1]-df_SF['transition_locations'][ind]))
        if (df_SF['transition_locations'][ind+1]-df_SF['transition_locations'][ind]) == 45:
            #print('# pulses correct')
            pass
    else:
        print('unknown failure')

8
5
original dataframe to work with, 7 transitions and their associated locations and sums: 
  transition                               transition_locations  \
0   111->101  [15, 375, 735, 1095, 1455, 1815, 2175, 2535, 2...   
1   101->100  [60, 420, 780, 1140, 1500, 1860, 2220, 2580, 2...   
2   100->110  [105, 465, 825, 1185, 1545, 1905, 2265, 2625, ...   
3   110->101  [150, 510, 870, 1230, 1590, 1950, 2310, 2670, ...   
4   101->110  [195, 555, 915, 1275, 1635, 1995, 2355, 2715, ...   
5   110->111  [240, 600, 960, 1320, 1680, 2040, 2400, 2760, ...   
6   111->100  [285, 645, 1005, 1365, 1725, 2085, 2445, 2805,...   
7   100->111  [330, 690, 1050, 1410, 1770, 2130, 2490, 2850,...   

                                            sumsTR_R      TR_R_avgs  \
0  [118288.0, 118385.0, 118305.0, 118316.0, 11831...  118343.500000   
1  [-116263.0, -116266.0, -116241.0, -116247.0, -... -116252.642857   
2  [339604.0, 339471.0, 339499.0, 339555.0, 33962...  339483.285714   
3  [65509.0, 65603.

In [63]:
cols = ['pulse', 'norms']
pulses = range(numRuns)
normsData = [pulses, HeNorms[0]]

#cols = ['transition', 'TR_R_avgs', 'TR_R stds']
#transSumsData = [transitions, transitionTR_RAvgs, transitionTR_Rstds]
print(len(pulses))
print(len(HeNorms[0]))

df_HE = pd.DataFrame({cols[0]: normsData[0],            
                   cols[1]: normsData[1]})

print(df_HE)

5000
5000
      pulse         norms
0         0  6.523311e+06
1         1  6.568455e+06
2         2  6.577309e+06
3         3  6.368333e+06
4         4  6.342620e+06
...     ...           ...
4995   4995  6.433206e+06
4996   4996  6.359883e+06
4997   4997  6.468415e+06
4998   4998  6.346348e+06
4999   4999  6.379081e+06

[5000 rows x 2 columns]


In [65]:
# for idx, df in enumerate([df_SF, df_HE]):
df_SF.to_hdf(savefilename + '.h5', f'df_0', mode='w') ## this "deletes" any previous data in the file name

for idx, df in enumerate([df_SF, df_HE]):
    df.to_hdf(savefilename + '.h5', f'df_{idx}', mode='a') # rerunning this without the above 'w' code will keep increasing file size

C:\Users\gabeo\AppData\Local\Temp\ipykernel_2068\3714383292.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['transition', 'transition_locations', 'sumsTR_R'], dtype='object')]

  df_SF.to_hdf(savefilename + '.h5', f'df_0', mode='w') ## this "deletes" any previous data in the file name
C:\Users\gabeo\AppData\Local\Temp\ipykernel_2068\3714383292.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['transition', 'transition_locations', 'sumsTR_R'], dtype='object')]

  df.to_hdf(savefilename + '.h5', f'df_{idx}', mode='a') # rerunning this without the above 'w' code will keep increasing file size


## end of file creation ##

In [67]:
## testing

df_SF2 = pd.read_hdf(savefilename + '.h5', key='df_0')
df_HE2 = pd.read_hdf(savefilename + '.h5', key='df_1')
print(df_SF2)
print(df_HE2)

     nicknames transition transition_locations  sumsTR_R      TR_R_avgs  \
0            0   111->101                   15  118288.0  118343.500000   
1            1   101->100                   60 -116263.0 -116252.642857   
2            2   100->110                  105  339604.0  339483.285714   
3            3   110->101                  150   65509.0   65628.071429   
4            4   101->110                  195   27146.0   27107.214286   
..         ...        ...                  ...       ...            ...   
106          2   100->110                 4785  339529.0  339483.285714   
107          3   110->101                 4830   65700.0   65628.071429   
108          4   101->110                 4875   27043.0   27107.214286   
109          5   110->111                 4920 -176513.0 -176458.000000   
110          6   111->100                 4965  -65730.0  -65738.717201   

      TR_R_stds  
0     47.433788  
1     10.444460  
2     83.418419  
3     59.654319  
4     53.

## above plus HeNorms should be saved to external file ##

In [71]:
# for ind in df.index:
#     print('transition: '+ str(df['nicknames'][ind]) + ' location: ' + str(df['transition_locations'][ind]))
#print(df['transition_locations'],df['nicknames'] )

test_arr = df_SF[['nicknames', 'transition_locations']].to_numpy().T
print((test_arr[1]))
print(len(test_arr[0]))
print(test_arr[1][0])
#print(np.array_split(test_arr[0], 14))
#testsums= np.random.rand(1,5000)
testsums= HeNorms/100000
# print(testsums[])
print(testsums)
i = 0
# print(testsums[0][30:45])
# print(testsums[0][test_arr[1][i]:test_arr[1][i+1]])

# # print((testsumarr))
# print(np.sum(testsums[0][35:75]))
#print(len(np.ndarray.flatten(np.asarray(sequence[1]))))

counter = 0
sequence = [[],[],[]]
seq = 0
smallerseq = []
smallersum = []
    
for i in range(len(test_arr[1])-(np.mod((len(test_arr[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
    counter = counter+1
    #print( i )
    #print(counter)
    if counter < 8:
        smallersum.append(np.sum(testsums[0][(test_arr[1][i])+5:(test_arr[1][i+1])])) ## minus first 5 to get rid of transitioning pulses
        smallerseq.append(test_arr[0][i+1])
        #print(smallerseq)
    elif counter == 8:
        seq = seq+1
        smallersum.append(np.sum(testsums[0][(test_arr[1][i])+5:(test_arr[1][i+1])]))
        smallerseq.append(test_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)   
        sequence[2].append(smallersum)
        smallerseq = []
        smallersum = []
        counter  = 0
        #print(test_arr[1][i+1])
        
        
# print(counter)
# print(len(testsumarr))
#print(len(testsumarr))
# print(len(sequence))
# print(len(sequence[0]))
# print(len(sequence[1]))

print(sequence)
print(len(sequence))
print(len(sequence[0]))
print(len(sequence[1]))
print(len(sequence[2]))

# # print((testsumarr))
# print(np.sum(testsums[0][35:75]))
# print(len(np.ndarray.flatten(np.asarray(sequence[1]))))

leftoversum = []
leftover = [[sequence[0][-1]+1],[]]

for i in range((len(test_arr[1])-(np.mod((len(test_arr[1])), 8))), len(test_arr[1])-1):
    counter = counter+1
    print( i )
    #print( i )
    #print(counter)
    if counter < 8:
        leftoversum.append(np.sum(testsums[0][(test_arr[1][i])+5:(test_arr[1][i+1])])) ## minus first 5 to get rid of transitioning pulses
        leftover[1].append(test_arr[0][i+1])
        #print(smallerseq)
    elif counter == 8:
        seq = seq+1
        leftoversum.append(np.sum(testsums[0][(test_arr[1][i])+5:(test_arr[1][i+1])]))
        smallerseq.append(test_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)       
        smallerseq = []
        counter  = 0
        #print(test_arr[1][i+1])

print(leftover)
print(leftoversum)

[15 60 105 150 195 240 285 330 375 420 465 510 555 600 645 690 735 780 825
 870 915 960 1005 1050 1095 1140 1185 1230 1275 1320 1365 1410 1455 1500
 1545 1590 1635 1680 1725 1770 1815 1860 1905 1950 1995 2040 2085 2130
 2175 2220 2265 2310 2355 2400 2445 2490 2535 2580 2625 2670 2715 2760
 2805 2850 2895 2940 2985 3030 3075 3120 3165 3210 3255 3300 3345 3390
 3435 3480 3525 3570 3615 3660 3705 3750 3795 3840 3885 3930 3975 4020
 4065 4110 4155 4200 4245 4290 4335 4380 4425 4470 4515 4560 4605 4650
 4695 4740 4785 4830 4875 4920 4965]
111
15
[[65.23310998 65.68454511 65.77308854 63.68332769 63.42620218 63.57585844
  63.2807009  62.70270064 63.64538949 63.70492592 65.19199636 63.31188541
  64.38952157 64.48744002 64.17154523 65.11883236 63.26531417 64.40708871
  64.78976835 63.65795981 63.09812772 63.85355735 64.02663332 62.90204306
  63.46294486 63.26074759 63.47743383 62.99766324 64.06784956 64.45432789
  63.9416363  64.69210701 63.20427613 63.21310394 63.77226657 64.96021085
  63.4802

In [ ]:
print(np.floor_divide(len(test_arr[1]),8))

In [189]:
print(len(test_arr[1])-(np.mod((len(test_arr[1])), 8)))

104


In [ ]:
## original that doesn't include leftover states

testsumarr = []
counter = 0
sequence = [[],[]]
seq = 0
smallerseq = []
leftover = []

for i in range(len(test_arr[1])-1):
    counter = counter+1
    #print(counter)
    if counter < 8:
        testsumarr.append(np.sum(testsums[0][(test_arr[1][i])+5:(test_arr[1][i+1])])) ## minus first 5 to get rid of transitioning pulses
        smallerseq.append(test_arr[0][i+1])
        #print(smallerseq)
    elif counter == 8:
        seq = seq+1
        testsumarr.append(np.sum(testsums[0][(test_arr[1][i])+5:(test_arr[1][i+1])]))
        smallerseq.append(test_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)        
        smallerseq = []
        counter  = 0
        #print(test_arr[1][i+1])
     
print(sequence)

# Stopped Here

Plot each channels' events

In [7]:
## normalizer

## Divide NaI and 6Li by 3He integral (normalize), then take integral of NaI and 6Li
baseL = 0
baseR = 1500
intgrL = 0 # bin 1800 looks OK for integral, but start at 0...
intgrR = 8999
HeintgrL = 7000 # 3He data is a different size
HeintgrR = 16000
#res =2.3 ms
sums = []
sums_norm = []
ys_basesub = []
ys_base_norm = [] 
legend = ['NaI', '6Li', '3He']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
#switchpulses = np.arange(230,575,43)
#print(switchpulses)
#p=8
pulse = 20
#print(pulse)
## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])

#%matplotlib qt
#for i in range(0,len(chan_enab)):
for i in (range(0,3)):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        if i==0: ## currently 3He is i=2. change if changing # of channels
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            tempsums.append(sum(tempys_basesub[j][HeintgrL:HeintgrR]))  ## Need to change sums array format if also doing LO/R
        else:
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            #print(np.mean(ys_arr[i][j][200:6000]))
            tempsums.append(sum(tempys_basesub[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(xs[i], tempys_basesub[j] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
 
        
## now normalize NaI and 6Li (can we do this in the forloop above?)

print(sums[2][pulse])
print((ys_basesub[1][pulse]))
print((ys_basesub[1][pulse]/sums[2][pulse]))

for i in (range(0,2)):
    tempys_base_norm = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        tempys_base_norm.append(ys_basesub[i][j]/(sums[2][j]/1000000))
        tempsums.append(sum(tempys_base_norm[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R    
    ys_base_norm.append(tempys_base_norm)
    sums_norm.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(xs[i], tempys_base_norm[j] , label=legend[i] +', normalized') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
        

#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")
print(len(xs[0]))
print(len(ys_arr[0][0]))
plt.axvline(xs[0][baseL], ls = '--')
plt.axvline(xs[0][baseR], ls = '--')
plt.axvline(xs[1][intgrL], ls = '--', c ='g')
plt.axvline(xs[1][intgrR], ls = '--', c ='g')
plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()

print('sums length:'+ str(len(sums)))
print(len(sums[2]))

#print(np.array(expectedSumsTR)+np.array(expectedSumsR))

print((sums[1][0]))
print((sums_norm[1][0]))
print((sums[1][0]/(sums[2][0]/1000000))) ##These are basically the same, so we can just use sums/normfactor
#plt.hist(sums, 100)
#plt.show(

6784184.000000795
[-0.066 -0.066 -0.066 ... -1.066 -1.066 -1.066]
[-9.72850972e-09 -9.72850972e-09 -9.72850972e-09 ... -1.57130172e-07
 -1.57130172e-07 -1.57130172e-07]
9000
9000
sums length:3
5000
-63365.88066668381
-9536.136309948082
-9536.136309950876


In [ ]:
expectedSumsLO = [sums[0][switchpulses[0]], sums[0][switchpulses[1]], sums[0][switchpulses[2]], sums[0][switchpulses[3]], sums[0][switchpulses[4]], sums[0][switchpulses[5]], sums[0][switchpulses[6]], sums[0][switchpulses[7]]]
expectedSumsTR = [sums[1][switchpulses[0]], sums[1][switchpulses[1]], sums[1][switchpulses[2]], sums[1][switchpulses[3]], sums[1][switchpulses[4]], sums[1][switchpulses[5]], sums[1][switchpulses[6]], sums[1][switchpulses[7]]]
expectedSumsR = [sums[2][switchpulses[0]], sums[2][switchpulses[1]], sums[2][switchpulses[2]], sums[2][switchpulses[3]], sums[2][switchpulses[4]], sums[2][switchpulses[5]], sums[2][switchpulses[6]], sums[2][switchpulses[7]]]
expectedSums = [np.array(expectedSumsLO), np.array(expectedSumsTR), np.array(expectedSumsR)]

print(expectedSumsTR)
print(expectedSumsR)
print('expected sums for TR+R: ')
print(np.array(expectedSumsTR)+np.array(expectedSumsR))
expectedTRandR = expectedSums[1]+expectedSums[2] # expected sums TR+R, by state
sumsTRandR = np.array(sums[1])+np.array(sums[2]) # all sums
print(len(sumsTRandR))


AllSwitches = []

for expectedsum in expectedTRandR:
    diff_arr = np.absolute(expectedsum - sumsTRandR)
    #print(len(diff_arr))
    found_sums =[]
    #nearest_i = diff_arr.argmin()
    for j in range(0,len(diff_arr)):
        if diff_arr[j] < 170:
            found_sums.append(j)
            #print(j)
    AllSwitches.append(np.array(found_sums))
            
        #found_sums = np.diff(np.absolute(diff_arr[j]) < 20, prepend=False)
        #print(diff_arr[j])
        #print(found_sums)
        #xing = np.argwhere(thresh_xing)[:,0]

#print(diff_arr)
for i in range(len(AllSwitches)):
    print(len(AllSwitches[i]))
print((AllSwitches))
#print(len(sums))
#print(expectedSumsTR)
#print(len(ys_arr[1]))
#plt.hist(sums, 100)
#plt.show(#switchs = [31, 74, 117, 160, 203, 246, 289, 332]
#%matplotlib notebook
baseL = 1
baseR = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])
sums = []
ys_basesub = [] 
legend = ['LO', 'TR', 'R']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
switchpulses = np.arange(230,575,43)
print(switchpulses)
p=8
pulse = switchpulses[p]
#print(pulse)

#%matplotlib qt
#for i in range(0,len(chan_enab)):
for i in range(0,3):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
        #print(np.mean(ys_arr[i][j][200:6000]))
        tempsums.append(sum(tempys_basesub[j][60:-1]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(xs[i], tempys_basesub[j] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   

#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('SF state transition') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")
print(len(xs[0]))
print(len(ys_arr[0][0]))
plt.axvline(xs[0][baseL], ls = '--')
plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[0][int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])], ls = '--', c ='m')
plt.axvline(xs[1][60], ls = '--', c ='c')
plt.axvline(xs[0][HeintgrL], ls = '--', c ='r')
plt.axvline(xs[0][HeintgrR], ls = '--', c ='r')
#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()

print('sums length:'+ str(len(sums)))
print(len(sums[2]))


expectedSumsLO = [sums[0][switchpulses[0]], sums[0][switchpulses[1]], sums[0][switchpulses[2]], sums[0][switchpulses[3]], sums[0][switchpulses[4]], sums[0][switchpulses[5]], sums[0][switchpulses[6]], sums[0][switchpulses[7]]]
expectedSumsTR = [sums[1][switchpulses[0]], sums[1][switchpulses[1]], sums[1][switchpulses[2]], sums[1][switchpulses[3]], sums[1][switchpulses[4]], sums[1][switchpulses[5]], sums[1][switchpulses[6]], sums[1][switchpulses[7]]]
expectedSumsR = [sums[2][switchpulses[0]], sums[2][switchpulses[1]], sums[2][switchpulses[2]], sums[2][switchpulses[3]], sums[2][switchpulses[4]], sums[2][switchpulses[5]], sums[2][switchpulses[6]], sums[2][switchpulses[7]]]
expectedSums = [np.array(expectedSumsLO), np.array(expectedSumsTR), np.array(expectedSumsR)]

print(expectedSumsTR)
print(expectedSumsR)
print('expected sums for TR+R: ')
print(np.array(expectedSumsTR)+np.array(expectedSumsR))
expectedTRandR = expectedSums[1]+expectedSums[2] # expected sums TR+R, by state
sumsTRandR = np.array(sums[1])+np.array(sums[2]) # all sums
print(len(sumsTRandR))


AllSwitches = []

for expectedsum in expectedTRandR:
    diff_arr = np.absolute(expectedsum - sumsTRandR)
    #print(len(diff_arr))
    found_sums =[]
    #nearest_i = diff_arr.argmin()
    for j in range(0,len(diff_arr)):
        if diff_arr[j] < 170:
            found_sums.append(j)
            #print(j)
    AllSwitches.append(np.array(found_sums))
            
        #found_sums = np.diff(np.absolute(diff_arr[j]) < 20, prepend=False)
        #print(diff_arr[j])
        #print(found_sums)
        #xing = np.argwhere(thresh_xing)[:,0]

#print(diff_arr)
for i in range(len(AllSwitches)):
    print(len(AllSwitches[i]))
print((AllSwitches))
#print(len(sums))
#print(expectedSumsTR)
#print(len(ys_arr[1]))
#plt.hist(sums, 100)
#plt.show(

## Add up all 5000 pulses


In [10]:
## old plotter

#switchs = [31, 74, 117, 160, 203, 246, 289, 332]
#%matplotlib notebook
baseL = 0
baseR = int(((preTime[1]-groupStart[1])*0.70)/chanDec[1])
sums = []
ys_basesub = [] 
legend = ['LO', 'TR', 'R']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
switchpulses = np.arange(83,430,45)
#print(switchpulses)
p=3
pulse = switchpulses[p]
#print(pulse)

#%matplotlib qt
#for i in range(0,len(chan_enab)):
for i in range(0,3):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
        #print(np.mean(ys_arr[i][j][200:6000]))
        tempsums.append(sum(tempys_basesub[j][60:-1]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(xs[i], tempys_basesub[j] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   

#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('SF state transition') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")
print(len(xs[1]))
print(len(ys_arr[0][0]))
plt.axvline(xs[1][baseL], ls = '--')
plt.axvline(xs[1][baseR], ls = '--')
#plt.axvline(xs[0][int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])], ls = '--', c ='m')
plt.axvline(xs[1][60], ls = '--', c ='r')
#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()
print(len(sums))
print((sums[1][20:30]))

# print('sums length:'+ str(len(sums)))
# print(len(sums[2]))


# expectedSumsLO = [sums[0][switchpulses[0]], sums[0][switchpulses[1]], sums[0][switchpulses[2]], sums[0][switchpulses[3]], sums[0][switchpulses[4]], sums[0][switchpulses[5]], sums[0][switchpulses[6]], sums[0][switchpulses[7]]]
# expectedSumsTR = [sums[1][switchpulses[0]], sums[1][switchpulses[1]], sums[1][switchpulses[2]], sums[1][switchpulses[3]], sums[1][switchpulses[4]], sums[1][switchpulses[5]], sums[1][switchpulses[6]], sums[1][switchpulses[7]]]
# expectedSumsR = [sums[2][switchpulses[0]], sums[2][switchpulses[1]], sums[2][switchpulses[2]], sums[2][switchpulses[3]], sums[2][switchpulses[4]], sums[2][switchpulses[5]], sums[2][switchpulses[6]], sums[2][switchpulses[7]]]
# expectedSums = [np.array(expectedSumsLO), np.array(expectedSumsTR), np.array(expectedSumsR)]

# print(expectedSumsTR)
# print(expectedSumsR)
# print('expected sums for TR+R: ')
# print(np.array(expectedSumsTR)+np.array(expectedSumsR))
# expectedTRandR = expectedSums[1]+expectedSums[2] # expected sums TR+R, by state
# sumsTRandR = np.array(sums[1])+np.array(sums[2]) # all sums
# print(len(sumsTRandR))


# AllSwitches = []

# for expectedsum in expectedTRandR:
#     diff_arr = np.absolute(expectedsum - sumsTRandR)
#     #print(len(diff_arr))
#     found_sums =[]
#     #nearest_i = diff_arr.argmin()
#     for j in range(0,len(diff_arr)):
#         if diff_arr[j] < 170:
#             found_sums.append(j)
#             #print(j)
#     AllSwitches.append(np.array(found_sums))
            
#         #found_sums = np.diff(np.absolute(diff_arr[j]) < 20, prepend=False)
#         #print(diff_arr[j])
#         #print(found_sums)
#         #xing = np.argwhere(thresh_xing)[:,0]

# #print(diff_arr)
# for i in range(len(AllSwitches)):
#     print(len(AllSwitches[i]))
# print((AllSwitches))
# #print(len(sums))
# #print(expectedSumsTR)
# #print(len(ys_arr[1]))
# #plt.hist(sums, 100)
# #plt.show()

351
351
3
[-4.0, -3.0, -4.0, -3.0, -4.0, -4.0, -4.0, -4.0, -4.0, -4.0]


In [9]:
baseL = 0
baseR = 1500
intgrL = 0 # bin 1800 looks OK for integral, but start at 0...
intgrR = 8999
HeintgrL = 7000 # 3He data is a different size
HeintgrR = 16000
#res =2.3 ms
sums = []
sums_norm = []
ys_basesub = []
ys_base_norm = [] 
legend = ['NaI', '6Li', '3He']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
#switchpulses = np.arange(230,575,43)
#print(switchpulses)
#p=8
pulse = 20
#print(pulse)
## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])

#%matplotlib qt
#for i in range(0,len(chan_enab)):
for i in (range(0,3)):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        if i==2: ## currently 3He is i=2. change if changing # of channels
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            tempsums.append(sum(tempys_basesub[j][HeintgrL:HeintgrR]))  ## Need to change sums array format if also doing LO/R
        else:
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            #print(np.mean(ys_arr[i][j][200:6000]))
            tempsums.append(sum(tempys_basesub[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    #for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        #plt.plot(xs[i], tempys_basesub[j] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
 
        
## now normalize NaI and 6Li (can we do this in the forloop above?)
## also add all 5000 pulses together

added_pulses = [] 

for i in (range(0,2)):
    tempys_base_norm = []
    tempsums =[]
    added_pulses.append(np.zeros(len(ys_basesub[i][0]))) ## make zeros array of length equal to window of cahnnel 
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        tempys_base_norm.append(ys_basesub[i][j]/(sums[2][j]/1000000))
        tempsums.append(sum(tempys_base_norm[j][intgrL:intgrR]))## Need to change sums array format if also doing LO/R    
        added_pulses[i] = added_pulses[i]+np.array(tempys_base_norm[j])
    ys_base_norm.append(tempys_base_norm)
    sums_norm.append(tempsums)
    #print(len(ys_basesub))
    #for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        #plt.plot(xs[i], tempys_base_norm[j] , label=legend[i] +', normalized') #+str(sums[1][j])) ## sums[j] will not work for more than just TR         
    plt.plot(xs[i], added_pulses[i] , label=legend[i] +', 5000 pulses added') 
    
#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")

plt.axvline(xs[0][baseL], ls = '--')
plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[1][intgrL], ls = '--', c ='g')
#plt.axvline(xs[1][intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()

print(len(added_pulses))
print(len(added_pulses[1]))


2
9000


In [10]:
T_0 = np.genfromtxt("/N/u/goterom/Quartz/Desktop/unpol_09919.csv", delimiter=",")
T_0 = T_0[2001:-1]
T_n = added_pulses[1][2001:-1]
print(len(T_0**2/T_n**2))
x_E = 3.895662*10**(-6)*(1/(xs[1][2001:-1]*10**(-9))**2)
print((x_E[380]))
print(xs[1][2400])
print(3.895662*10**(-6)*(1/(512*10**(-9)))**2)
print((512*10**(-9)))


P_E = np.sqrt(np.absolute(1 - (T_0**2/T_n**2)))

plt.plot(x_E[380:-1], P_E[380:-1]) 
    
#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('3He Polarization P(E)') 
plt.xlabel("E [eV]")
plt.ylabel("P(E)")

#plt.axvline(xs[0][baseL], ls = '--')
#plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[1][intgrL], ls = '--', c ='g')
#plt.axvline(xs[1][intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


6998
102.37440620551811
204800.0
14860771.179199215
5.12e-07


Now get the final sum over all pulses

In [9]:
## sum the sums... Only 6Li is important for this, but keeping it general

legend = ['NaI', '6Li']
sumTot = []
for i in (range(0,2)):
    tempsum = []
    tempsum.append(sum(sums[i]/(np.array(sums[2])/1000000)))
    sumTot.append(tempsum)
    
print(np.mean(sums[2]))
    
print(sumTot)
print('Sum for all spin states of run' + run_num + ': ' + str(sumTot[1][0]))

6652627.0427999925
[[726399252.2521336], [-47553734.12199138]]
Sum for all spin states of run12400: -47553734.12199138


Plot in terms of eV?

In [26]:
## Need to look after 0. Only look at resonance regions.
## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])
testeV = 0.5*(1.6749286*10**(-27))*(27.3/(np.array(xs[1][2001:])*10**9))**2*(6.24151*10**18)
for i in (range(0,2)):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        if i==2: ## currently 3He is i=2. change if changing # of channels
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            tempsums.append(sum(tempys_basesub[j][HeintgrL:HeintgrR]))  ## Need to change sums array format if also doing LO/R
        else:
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            #print(np.mean(ys_arr[i][j][200:6000]))
            tempsums.append(sum(tempys_basesub[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(testeV, tempys_basesub[j][2001:] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   

plt.title('Detector signals') 
plt.xlabel("eV")
plt.ylabel("ADC")
print(len(xs[0]))
print(len(ys_arr[0][0]))
#plt.axvline(xs[0][baseL], ls = '--')
#plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(testeV[intgrL], ls = '--', c ='g')
#plt.axvline(testeV[intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()
print(xs[1][2001:])
print(testeV)

9000
9000
[5.120000e+02 1.024000e+03 1.536000e+03 ... 3.582464e+06 3.582976e+06
 3.583488e+06]
[1.48607711e-29 3.71519278e-30 1.65119679e-30 ... 3.03541166e-37
 3.03454421e-37 3.03367714e-37]


In [8]:
sumsLO = []
sumsTR = []
sumsR = []

ys_basesub = []
legend = ['LO', 'TR', 'R']
statesID = ['111', '101', '100', '110','101','110','111','100','111']
switchpulses = np.arange(230,575,43)

for j in switchpulses:
    ys_basesub.append(ys_arr[0] - np.mean(ys_arr[0][j][baseL:baseR]))
    sumsLO.append(sum(ys_basesub[0][j][60:-1]))
    #print((len(ys_basesub)))
    ys_basesub = []
for j in switchpulses:
    ys_basesub.append(ys_arr[1] - np.mean(ys_arr[1][j][baseL:baseR]))
    sumsTR.append(sum(ys_basesub[0][j][60:-1]))
    #print((len(ys_basesub)))
    ys_basesub = []
for j in switchpulses:
    ys_basesub.append(ys_arr[2] - np.mean(ys_arr[2][j][baseL:baseR]))
    sumsR.append(sum(ys_basesub[0][j][60:-1]))
    #print((len(ys_basesub)))
    ys_basesub = []
    
sumsLO =np.array(sumsLO)
sumsTR =np.array(sumsTR)
sumsR =np.array(sumsR)
    
print(sumsLO)
print(sumsTR)
print(sumsR)
print(sumsTR+sumsR)

[974515.00199964 936179.22399994 975085.73533334 946074.0853335
 956966.17666671 940943.62866666 963640.02533324 945226.96800023
 993454.70400038]
[-63718.27533335 -62157.608      -63476.67466667 -62421.94266669
 -61866.         -61884.34933335 -63441.75600001 -64378.96666666
 -62743.83733334]
[37273269.79600207 37690246.97534942 37648681.45802363 37170611.38800599
 36896463.00266942 37285983.69603391 37598406.80201846 37789491.5986473
 37631863.84996781]
[37209551.52066872 37628089.36734942 37585204.78335696 37108189.44533931
 36834597.00266942 37224099.34670056 37534965.04601846 37725112.63198064
 37569120.01263448]


In [9]:
##Threshold testing
TRThresh = 32 ## Note the threshold can change from statechange to statechange. Need to figure out a good value.
RThresh = 30
ys_basesub = []
where = []
pulseswitch_list = []
zeros=0
zeros_arr = []
switchbin = []
States_full = [zeros_arr,switchbin]

#for i in range(0,len(chan_enab)):
for i in range(1,2):
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])):
    #for j in range(s, t):
        ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        thresh_xing = np.diff(np.absolute(ys_basesub[j]) > TRThresh, prepend=False)
        xing = np.argwhere(thresh_xing)[:,0]
        #print(len(xing))
        if len(xing)== 0:
            #pulseswitch_list.append(0)  ## can now get rid of this, but illustrrates non-switch and switch pulses
            zeros+=1
        else:
            #pulseswitch_list.append(xing[0])  ## can now get rid of this, but illustrrates non-switch and switch pulses
            switchbin.append(j)
            zeros_arr.append(zeros)
            zeros=0
        
#for pulse in pulseswitch_list:
#    #print(pulse)
#    if pulse==0:
#        #print(zeros[0])
#        zeros+=1
#    else:
#        switchbin.append(pulse)
#        zeros_arr.append(zeros)
#        zeros=0
        
States_full = np.array(States_full).T
            
for state in States_full[1:]:
    if state[0]!=42:
        print('SF ERROR!')
    else:
        print('SF OK')
        #where.append(np.where(ys_basesub[j] > TRThresh))
        #print(np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j]))
        #sums.append(sum(ys_basesub[i][j][9000:-1]))
        #plt.plot(xs[i], ys_basesub[i][j], label=legend[i]+', Integral = '+str(sums[i]))

#print(pulseswitch_list)
print(States_full)
#print(zeros)

SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
[[  15   15]
 [  42   58]
 [  42  101]
 [  42  144]
 [  42  187]
 [  42  230]
 [  85  316]
 [  42  359]
 [  42  402]
 [  42  445]
 [  42  488]
 [  42  531]
 [  42  574]
 [  85  660]
 [  42  703]
 [  42  746]
 [  42  789]
 [  42  832]
 [  42  875]
 [  42  918]
 [  85 1004]
 [  42 1047]
 [  42 1090]
 [  42 1133]
 [  42 1176]
 [  42 1219]
 [  42 1262]
 [  85 1348]
 [  42 1391]
 [  42 1434]
 [  42 1477]
 [  42 1520]
 [  42 1563]
 [  42 1606]
 [  85 1692]
 [  42 1735]
 [

BELOW IS TESTING

In [9]:
#print(np.mean(ys_arr[0][0:6000]))
ys_basesub = []

#for i in range(0,len(chan_enab)):
#    ys_basesub.append(ys_arr[i] - np.mean(ys_arr[i][0:6000]))
#    print(np.mean(ys_arr[i][0:6000]))

for i in range(0,len(chan_enab)):
#for i in range(0,1):
    s=pulse
    t=s+1
    #for j in range(0, len(ys_arr[i])):
    for j in range(s, t):
        ys_basesub.append(ys_arr[i] - np.mean(ys_arr[i][j][300:6000]))
        print(np.mean(ys_arr[i][j][300:6000]))
        sums.append(sum(ys_arr[i][j]))
        plt.plot(xs[i], ys_basesub[i][j], label=legend[i]+', Integral = '+str(sums[i]))

#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('SF state transition '+statesID[p]+' -> '+statesID[p+1])
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")
#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()
    
print(len(ys_basesub[0][26][0:6000]))

1062.9626315789474
1080.0126315789473
1092.0129824561404
6000


In [10]:
#switchs = [31, 74, 117, 160, 203, 246, 289, 332]
#%matplotlib notebook
sums = []
switchs = np.arange(31,1923,43)
print(switchs)
ys_basesub = []

for i in range(0,len(chan_enab)):
    ys_basesub.append(ys_arr[i] - np.mean(ys_arr[0][0:6000]))
    
#%matplotlib qt
#for i in range(0,len(chan_enab)):
#for i in range(2,5):
#    s=375
#    t=s+1
#    #for j in range(0, len(ys_arr[i])):
#    for j in range(s, t):
#        plt.plot(xs[i], ys_arr[i][j])
#for i in range(2,5):
#    s=332
#    t=s+1
#    #for j in range(0, len(ys_arr[i])):
#    for j in range(s, t):
#        plt.plot(xs[i], ys_arr[i][j])
for i in range(4,5):
    for j in switchs:
        plt.plot(xs[i], ys_arr[i][j])
        sums.append(sum(ys_arr[i][j]))
#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.xlabel("time from trigger (ns)")
plt.show()
print(sums)
print(len(ys_arr[1]))
plt.hist(sums, 100)
plt.show()

[  31   74  117  160  203  246  289  332  375  418  461  504  547  590
  633  676  719  762  805  848  891  934  977 1020 1063 1106 1149 1192
 1235 1278 1321 1364 1407 1450 1493 1536 1579 1622 1665 1708 1751 1794
 1837 1880]


IndexError: list index out of range

In [ ]:
pulse = 289
for i in range(4,5):
    for j in switchs:
        plt.plot(xs[i], ys_arr[i][pulse])
print(ys_arr[4][pulse])
print(sum(ys_arr[4][pulse]))

summation for all channels

In [ ]:
NaIthres=3500
Li6thres=200
Hethres=0
NaIsign="pos"
Li6sign="neg"
Hesign=""
threshold_array=[NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,
                 NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,
                NaIthres,NaIthres,NaIthres,NaIthres,Li6thres,Hethres]
threshold_sign=[NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,
                NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,
                NaIsign,NaIsign,Li6sign,Hesign]
xCrosses=[]#outer array is crossing arrays for given channel, inner array is crossing for each event
modeCrosses=[] #single array with one value per channel
xValues=[]
xtimems=[]

extendedRange=3
for i in range(0,len(chan_enab)):
    xCrosses.append([])
    xValues.append([])
    for j in ys_arr[i]:
        newj = np.array(j)
        if threshold_sign[i] == "pos":
            thresh_xing = np.diff(newj > threshold_array[i], prepend=False)
        elif threshold_sign[i] == "neg":
            thresh_xing = np.diff(newj < threshold_array[i], prepend=False)
        xing = np.argwhere(thresh_xing)[:,0]
        xCrosses[i].append(xing[0])
    modeCrosses.append(st.mode(xCrosses[i])) #find the most typical crossing value for each channel
    xValues[i].append(np.arange(len(ys_arr[i][0])+2*extendedRange)-modeCrosses[i])
    xtimems.append(xValues[i][-1]*resolution[i]*10**(-6))
print(len(xtimems[1]))
print(modeCrosses)
print(xValues[0])

                       

In [ ]:
#print(xCrosses)
extendedRange = 3


time_array=[]
time_nparray=[]
failedRuns=[]
for i in range(0,len(chan_enab)):
#for i in range(0,2):
    time_array.append(np.zeros(len(ys_arr[i][0])+2*extendedRange))
    time_nparray.append([])
    for j in range(0, len(ys_arr[i])):
        shiftedWave = ys_arr[i][j][:] - np.mean(ys_arr[i][j][0:int(xing[0]*0.80)]) #this subtracts the baseline -
        #baseline is based on the first 80% of samples before the threshold crossing
        offset = xCrosses[i][j]-modeCrosses[i]
        if (abs(offset) < extendedRange+1):
            for k in range(len(shiftedWave)):
                time_array[i][k-offset+extendedRange]=time_array[i][k-offset+extendedRange]+shiftedWave[k]
                #may need to switch this to subtracting offset
        else:
            failedRuns.append([i,j])
    time_nparray[i]=np.array(time_array[i])
#print(xCrosses)
print(len(time_nparray[0]))

In [ ]:
for i in range(0,len(chan_enab)):
#for i in range(0,1):
    plt.plot(xtimems[i],time_nparray[i])
    plt.xlabel('Time [ms]')
    plt.ylabel('counts (arb)')
    #plt.xlim(-0.1,0.2)
    #plt.ylim(-600000,0)
plt.show()

In [ ]:

d = {"right": [0,1,8,9,10,11,14,15,16,17,18,19],
     "left": [2,3,4,5,6,7,12,13,20,21,22,23],
     "up": [12,13,14,15,16,17,18,19,20,21,22,23],
     "down": [0,1,2,3,4,5,6,7,8,9,10,11],
     "corner": [0,3,6,9,12,15,18,21],
     "noncorner": [1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23],
     "top": [0,1,2,3,4,11,12,13,14,15,16,23],
     "bottom": [5,6,7,8,9,10,17,18,19,20,21,22],
     "Li6": [24],
     "NaI": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
     "He3": [25]
    }

In [ ]:
type1="up"
type2="down"
type1_array=np.zeros(len(ys_arr[0][0]))
type2_array=np.zeros(len(ys_arr[0][0]))
for i in d[type1]: 
    type1_array = np.add(type1_array, time_nparray[i])
    
for i in d[type2]:
    type2_array = np.add(type2_array, time_nparray[i])
    plt.title(type1 + " in blue, " + type2 + " in red")
plt.plot(xtimems[0], type1_array/len(d[type1]),'b', lw=1)
plt.plot(xtimems[0], type2_array/len(d[type2]),'r',lw=1)
plt.show()

In [6]:
## old read data

ys_arr = []
ETTT_arr = []
eventcount_arr = []

for i in range(0,len(chan_enab)):
    eventCount = 0
    byteCounter = 0
    ys_arr.append([])
    eventcount_arr.append([])
    
    #byte counter is really 2bytecounter, lol
    while byteCounter < fileLength[i]:
        if byteCounter == 0:
            ETTT_arr.append([])
            eventcount_arr.append([])
            ETTT_arr[i].append(TTT[i])
            eventcount_arr[i].append(eventCounter[i])
            byteCounter = 20
        else:
            ETTT_arr[i].append(read_data[i][byteCounter]+(read_data[i][byteCounter+1]<<16)+(read_data[i][byteCounter+2]<<32))
            eventcount_arr[i].append(read_data[i][byteCounter+4]+(read_data[i][byteCounter+5]<<16))
            byteCounter += 6
        for j in range(0, numSamples[i]):
            if j == 0:
                ys_arr[i].append([])
            ys_arr[i][eventCount].append(read_data[i][byteCounter]) 
            byteCounter += 1
        eventCount += 1
#read_data.clear()
    #times_arr.append(list(map(list, zip(*headers_arr[i][1:])))[0])

KeyboardInterrupt: 